In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

In [12]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client)

In [13]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

get_latest_news 
    Fetches the latest news headlines from a supported static news source.

    This tool currently supports the following sources:
    - 'npr'     → National Public Radio
    - 'bbc'     → BBC News
    Args:
        source (str): The news source to fetch headlines from. Must be one of:
                    'npr', 'bbc', or 'reuters'. Case-insensitive.

    Returns:
        str: A plain text string with the top 10 headlines from the selected source,
            separated by newlines. If the source is unsupported or an error occurs,
            a corresponding message is returned.

    Example:
        >>> get_latest_news("bbc")
    
get_wikipedia_summary 
    Fetches the first paragraph summary of a given topic from Wikipedia.

    Parameters:
        topic (str): The topic to search for (e.g., "machine learning").

    Returns:
        str: The summary paragraph from the topic's Wikipedia page,
             or an error message if not found.

    Example:
        >>> ge

## System Prompt for Sqlite-Sever

In [5]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with Our Database
"""

## System Prompt for News-Server

In [14]:
SYSTEM_PROMPT = """\
You are an AI assistant with access to these tools:

1. Wikipedia Tools:
   - get_wikipedia_summary: Get summaries of topics from Wikipedia

2. News Tools:
   - get_latest_news: Get current news headlines from NPR or BBC

3. Stock Tools:
   - get_stock_news: Get recent news about specific stocks

Use these tools when appropriate to answer user questions.
"""

In [15]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with News Sources.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

In [16]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [17]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [18]:
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  get me the latest news from bbc
Calling tool get_latest_news with kwargs {'source': 'bbc'}
Tool get_latest_news returned meta=None content=[TextContent(type='text', text="- Gazans wait for aid as UN says supplies sitting in warehouse despite Israel easing blockade\n- Canada in talks with US to join Trump's Golden Dome defence system\n- India's Banu Mushtaq makes history with International Booker win\n- School bus attack kills at least five in Pakistan\n- R&B star Chris Brown freed on £5m bail by London court\n- Canada in talks with US to join Trump's Golden Dome defence system\n- Six Ukrainian soldiers killed in Russian strike on training exercise\n- British soldiers make Everest history using new method\n- An island called Hope is standing up to Beijing in the South China Sea\n- India's Banu Mushtaq makes history with International Booker win", annotations=None)] isError=False
Agent:  Here are the latest news headlines from BBC:

* Gazans wait for aid as UN says supplies sittin